## Morhunenko Mykola. Fourth variant. "Peninsula" region
`descriptive` file was taken as a template

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from datetime import datetime
import matplotlib.pyplot as plt
import pyflux as pf
import statsmodels as ss
import statsmodels.api as sm
import seaborn as sns
import sys
import warnings
import matplotlib.pyplot as plt
from statsmodels.tsa.seasonal import seasonal_decompose
from statsmodels.tsa.api import ExponentialSmoothing
from statsmodels.tsa.stattools import adfuller
import statsmodels.tsa.api as smt
from sklearn.metrics import mean_squared_error as mse
from fbprophet import Prophet
from xgboost import XGBRegressor
from pandas import to_datetime
from fbprophet.diagnostics import cross_validation
from fbprophet.diagnostics import performance_metrics
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense, Dropout, TimeDistributed, Conv1D, MaxPooling1D, Flatten, RepeatVector
from keras.layers import LSTM
from keras.optimizers import Adam, Adagrad
from hyperas import optim
from hyperas.distributions import choice, uniform
from hyperopt import Trials, STATUS_OK, tpe, rand

%matplotlib inline
matplotlib.rcParams['figure.dpi']= 100
matplotlib.rcParams['figure.figsize'] = 15, 5

In [2]:
REGIONS = [['JH', 'Johor', 'JH'], 
           ['PH', 'Pahang', 'PH'],
           ['PRK', 'Perak', 'PRK'], 
           ['OtherPEN', 'Other Pen. States', 'OtherPEN'],
           ['PEN', 'Peninsula', 'Pmalay'],
           ['SBH', 'Sabah', 'SBH'],
           ['SWK', 'Sarawak', 'SWK']]

In [4]:
production = pd.read_csv('palm_data/production_good.csv')
rainfall = pd.read_csv('palm_data/rainfall_good.csv')
area = pd.read_csv('palm_data/area_good.csv')

In [ ]:
RAINFALL_LAGS = [6, 7, 8, 9, 10, 11, 12]
PRODUCTION_LAGS = [6, 7, 8, 9, 10, 11, 12]

In [ ]:
def process_data(production, rainfall, area, REGIONS, PRODUCTION_LAGS, RAINFALL_LAGS):
    output = {}
    for i, region in enumerate(REGIONS):
        data = production[production.Region == region[0]]
        data = pd.merge(data[['Year', 'Month', 'Production', 'Diff_production']],
                        rainfall[rainfall.Region == region[1]][['Year', 'Month', 'Rainfall']], 
                        on=['Year', 'Month'], how='left')

        data = pd.merge(data, area[area.Region == region[2]][
            ['Year', 'Area_ma', 'Area_npa', 'Area_rpa', 'Area_New', 'Area_ma_new']], on='Year', how='left')
            
        data['Year'] = data['Year'].astype(int)
        data['Month'] = data['Month'].astype(int)
        data.reset_index(inplace=True)

        data.set_index([pd.to_datetime(['{0}-{1}-01'.format(x, y) for (x, y) in zip(data.Year, data.Month)])],
                       inplace=True)

        data.drop(['index', 'Year', 'Area_New', 'Diff_production'], axis=1, inplace=True)
        data['Time'] = np.arange(len(data))

        for lag in RAINFALL_LAGS:
            temp = np.concatenate((np.array([np.nan for _ in range(lag)]), data.Rainfall.values[:-lag]))
            data['Rainfall_{0}'.format(lag)] = temp

        for lag in PRODUCTION_LAGS:
            temp = np.concatenate((np.array([np.nan for _ in range(lag)]), data.Production.values[:-lag]))
            data['Production_{0}'.format(lag)] = temp
        
        PRODUCTION = data.Production
        data.drop(['Production'], axis=1, inplace=True)

        data.fillna(data.mean(), inplace=True)

        #  And finally drop rainfalls
        data.drop(['Rainfall'], axis=1, inplace=True)
        ### HERE I CAN ADD FEATURE ENGINEERING!!!

        #  And clip first year
        for col in data.columns:
            data['_'.join([region[1], str(col)])] = data[col]
            data.drop([col], axis=1, inplace=True)
        
        output[region[1]] = (data[max(PRODUCTION_LAGS):], PRODUCTION[max(PRODUCTION_LAGS):])
    return output


In [ ]:
DATA = process_data(production, rainfall, area, REGIONS, PRODUCTION_LAGS, RAINFALL_LAGS)

In [ ]:
DATA.keys()

In [ ]:
DATA=DATA['Peninsula']

In [ ]:
time = np.arange(len(DATA[1]))

#### Stationarity
Time-series property that indicates that the chosen Time Series has constant mean, variance, autocorrelation structure and have no periodic components over time. To get a stationary time series, we can split it into components (trend, seasonality and residuals) and take that residual. Thay can be stationary, but to be sure, we have to test it (for example, use Augmented Dickey-Fuller test).
#### Differencing
One more method to make a series stationary. If we have lagged data and can see autocorrelations in it, we can subtract from our series the same series, but lagged by one. More simple: from every element $E_{t}$ subtract $E_{t-1}$
#### Moving Average, Exponential Smoothing
Both - techniques to extract useful patterns from the series. 
Moving average - we chose a window and simply slide over the series with that window and calculate average. Can be used to find a trend, seasonality and a little bit of noise.
Exponential Smoothing - when our weights in the observation history are different. For example, nearest data have a bigger impact than older

# Additive VS multiplicative

In [ ]:
DATA[1].plot(figsize=(15,5))

### Additive 

We have a time series, that can be either additive ot multiplicative. This cpecific looks like additive, but the task is to make all on both data. Firstle, let's split the series on the trend, seasonality and residuals. As far as we have monthly data, period have to be 12. 

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

ss_decomposition = seasonal_decompose(x=DATA[1], model='additive', period=12)
estimated_trend_add = ss_decomposition.trend
estimated_seasonal_add = ss_decomposition.seasonal
estimated_residual_add = ss_decomposition.resid

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False)
fig.set_figheight(10)
fig.set_figwidth(15)

axes[0].plot(DATA[1], label='Original')
axes[0].legend(loc='upper left');

axes[1].plot(estimated_trend_add, label='Trend')
axes[1].legend(loc='upper left');

axes[2].plot(estimated_seasonal_add, label='Seasonality')
axes[2].legend(loc='upper left');

axes[3].plot(estimated_residual_add, label='Residuals')
axes[3].legend(loc='upper left');

In [ ]:
estimated_residual_add = estimated_residual_add.dropna()

In [ ]:
estimated_residual_add

In [ ]:
pd.Series(estimated_residual_add).hist();

### Multiplicative

In [ ]:
from statsmodels.tsa.seasonal import seasonal_decompose

ss_decomposition = seasonal_decompose(x=DATA[1], model='multiplicative', period=12)
estimated_trend_mul = ss_decomposition.trend
estimated_seasonal_mul = ss_decomposition.seasonal
estimated_residual_mul = ss_decomposition.resid

In [ ]:
fig, axes = plt.subplots(4, 1, sharex=True, sharey=False)
fig.set_figheight(10)
fig.set_figwidth(15)

axes[0].plot(DATA[1], label='Original')
axes[0].legend(loc='upper left');

axes[1].plot(estimated_trend_mul, label='Trend')
axes[1].legend(loc='upper left');

axes[2].plot(estimated_seasonal_mul, label='Seasonality')
axes[2].legend(loc='upper left');

axes[3].plot(estimated_residual_mul, label='Residuals')
axes[3].legend(loc='upper left');

In [ ]:
estimated_residual_mul = estimated_residual_mul.dropna()


In [ ]:
estimated_residual_mul

In [ ]:
pd.Series(estimated_residual_mul).hist();

In [ ]:
adf_add, pvalue_add, usedlag_add, nobs_add, critical_values_add, icbest_add = adfuller(estimated_residual_add)
adf_mul, pvalue_mul, usedlag_mul, nobs_mul, critical_values_mul, icbest_mul = adfuller(estimated_residual_mul)

In [ ]:
print(adf_add, adf_mul)

In [ ]:
print(pvalue_add, pvalue_mul)

In [ ]:
print(critical_values_add, "\n",critical_values_mul)

using that p-values, we can decide, which model is better to choose, but let's continue with both

## Remove autocorelation with differencing


One more way to get stationary series is to aplly difference method. it can be applied to a lagged data, and here we have such data.

In [ ]:
difference = DATA[1] - DATA[1].shift(1)

In [ ]:
estimated_residual_dif = pd.Series(difference, index=DATA[1][1:].index)

In [ ]:
# plt.plot(np.arange(len(difference)), difference)
estimated_residual_dif.plot()

In [ ]:
adf_dif, pvalue_dif, usedlag_dif, nobs_dif, critical_values_dif, icbest_dif = adfuller(estimated_residual_dif)

In [ ]:
print(adf_dif)

In [ ]:
print(pvalue_dif)

In [ ]:
print(critical_values_dif)

now if we compare pvalues of decomposition and differencing methods, it is better to take the assumption that the series is additive and use decomposition. but for now we have three series: estimated_residual_add, estimated_residual_dif, estimated_residual_mul and will work with all of them

# Moving Average, Exponential Smoothing

### Mooving Average

In [ ]:
DATASET = [estimated_residual_add, estimated_residual_mul, estimated_residual_dif]

ts_moving_avg_right = DATA[1].rolling(12, center=False).mean()
plt.plot(ts_moving_avg_right)

Let's get our stationary series, residuals, using MA result

In [ ]:
ts_ma_diff = DATA[1] - ts_moving_avg_right
ts_ma_diff.plot()

In [ ]:
DATASET.append(ts_ma_diff.dropna())

In [ ]:
def stationarity_test(ts):
    # only now I found the stationarity test in the template file, so I will use it
    
    rolmean = ts.rolling(12).mean()
    rolstd = ts.rolling(12).std()

    orig = plt.plot(ts, color='blue',label='Original')
    mean = plt.plot(rolmean, color='red', label='Rolling Mean')
    std = plt.plot(rolstd, color='black', label = 'Rolling Std')

    # Dickey-Fuller test:
    print("Results of Dickey-Fuller Test:")
    dftest = adfuller(ts, autolag='AIC')
    print(dftest)     
    dfoutput = pd.Series(dftest[0:4], index=['Test Statistic','p-value','#Lags Used','Number of Observations Used'])
    for key,value in dftest[4].items():
        dfoutput['Critical Value (%s)'%key] = value
    print(dfoutput)

In [ ]:
stationarity_test(DATASET[-1])

we have one more candidate to be our stationary series according to the test (but easy to see seasonality there..)

### Exponential Smoothing

In [ ]:
expwighted_avg = DATA[1].ewm(span=12).mean()

In [ ]:
stationarity_test(expwighted_avg)

In [ ]:
DATA[1].plot(color="blue", label="Original")
expwighted_avg.plot(color="red", label="EWMA")

In [ ]:
ts_exp_diff = DATA[1] - expwighted_avg
ts_exp_diff.plot()

In [ ]:
stationarity_test(ts_exp_diff)

and one more candidate for predictions

In [ ]:
DATASET.append(ts_exp_diff)

In [ ]:
len(DATASET)
DATASET = [each.dropna() for each in DATASET]

#### NOW WE HAVE FIVE STATIONARY  SERIES
all of them are in the DATASET array. 
- DATASET[0] - residuals got after splitting input data using decomposition and suppose that it is additive
- DATASET[1] - residuals got after splitting input data using decomposition and suppose that it is multiplictive
- DATASET[2] - series got using differencing
- DATASET[3] - series got after substracting all features that was found with MA
- DATASET[4] - series got after substracting all features that was found with EWMA
# **ALL next plots will be in this order **


## Simple Average (just for training)

In [ ]:

for counter in range(len(DATASET)):
    train = DATASET[counter][:-112]
    test = DATASET[counter][-113:-101]
    trend_seasonal_avg = np.mean(DATASET[counter])
    time = np.arange(len(DATASET[counter]) - 100)
    simple_avg_preds = np.full(shape=12, fill_value=trend_seasonal_avg, dtype='float')
    plt.plot(time[:-12], train, 'b--', label="train")
    plt.plot(time[-12:], test, color='orange', linestyle="--", label="test")
    plt.plot(time[-12:], simple_avg_preds, 'r--', label="predictions")
    plt.legend(loc='upper left')
    plt.title("Simple Average Smoothing")
    plt.grid(alpha=0.3)
    plt.show();
    
    print("MSE = {}".format(mse(test, simple_avg_preds)))

useless (especcialy in this case)

### Triple Exponential

In [ ]:
i = 0
for counter in range(len(DATASET)):
    i += 1
    train = DATASET[counter][:-112]
    test = DATASET[counter][-113:-101]
    time = np.arange(len(DATASET[counter]) -100)
    if (i == 2):
        triple = ExponentialSmoothing(train,
                              trend="multiplicative",
                              seasonal="multiplicative",
                              seasonal_periods=12).fit(optimized=True)
    else:
        triple = ExponentialSmoothing(train,
                              trend="additive",
                              seasonal="additive",
                              seasonal_periods=12).fit(optimized=True)
    triple_preds = triple.forecast(len(test))
    
    plt.plot(time[:-12], train, 'b--', label="train")
    plt.plot(time[-12:], test, color='orange', linestyle="--", label="test")
    plt.plot(time[-12:], triple_preds, 'r--', label="predictions")
    plt.legend(loc='upper left')
    plt.title("Triple Exponential Smoothing")
    plt.grid(alpha=0.3);
    plt.show()
    
    print("MSE = {}".format(mse(test, triple_preds)))

we can see that this approach gives us not bad predictions

# AR, MA, ARMA, ARIMA, (SARIMAX MODULE)

In [ ]:
def MAPE(data, fit):
    return 100*(np.average(abs((fit-data)/data)))

In [ ]:
# define helper plot function for visualization

def plots(data, lags=None):
    layout = (1, 3)
    raw  = plt.subplot2grid(layout, (0, 0))
    acf  = plt.subplot2grid(layout, (0, 1))
    pacf = plt.subplot2grid(layout, (0, 2))
    
    data.plot(ax=raw)
    smt.graphics.plot_acf(data, lags=lags, ax=acf)
    smt.graphics.plot_pacf(data, lags=lags, ax=pacf)
    sns.despine()
    plt.tight_layout()
    plt.show()
    
    

In [ ]:
for each in DATASET:
    plots(each, lags=48);

according to autocorelation function results, p=3 (4th zeo) <br>
according to partial autocorelation function results, q=2 (3d zero)

# AR


In [ ]:

for dt in DATASET:

    # fit SARIMA monthly based on helper plots
    sar = sm.tsa.statespace.SARIMAX(dt, 
                                    order=(3,0,0),  
                                    trend='c').fit()

    print(sar.plot_diagnostics())
    plots(dt, lags=48)
    print(sar.summary())
    res = sar.predict(start = 0, end= len(dt), dynamic=False)
    res.plot()
    dt.plot()
    
    print("MSE = {}".format(mse(dt, res[:-1])))
    print("MAPE = {}".format(MAPE(dt, res[:-1])))

# MA

In [ ]:

for dt in DATASET:

    # fit SARIMA monthly based on helper plots
    sar = sm.tsa.statespace.SARIMAX(dt, 
                                    order=(0,0,2),  
                                    trend='c').fit()

    print(sar.plot_diagnostics())
    plots(dt, lags=48)
    print(sar.summary())
    res = sar.predict(start = 0, end= len(dt), dynamic=False)
    res.plot()
    dt.plot()
    
    print("MSE = {}".format(mse(dt, res[:-1])))
    print("MAPE = {}".format(MAPE(dt, res[:-1])))

# ARMA

In [ ]:

for dt in DATASET:

    # fit SARIMA monthly based on helper plots
    sar = sm.tsa.statespace.SARIMAX(dt, 
                                    order=(3,0,2),  
                                    trend='c').fit()

    print(sar.plot_diagnostics())
    plots(dt, lags=48)
    print(sar.summary())
    res = sar.predict(start = 0 , end= len(dt), dynamic=False)
    res.plot()
    dt.plot()
    
    print("MAPE = {}".format(MAPE(dt, res[:-1])))
    print("MSE = {}".format(mse(dt, res[:-1])))

# ARIMA

In [ ]:


for dt in DATASET:

    # fit SARIMA monthly based on helper plots
    sar = sm.tsa.statespace.SARIMAX(dt, 
                                    order=(3,1,2),  
                                    trend='c').fit()

    print(sar.plot_diagnostics())
    plots(dt, lags=48)
    print(sar.summary())
    res = sar.predict(start = 0 , end= len(dt), dynamic=False)
    res.plot()
    dt.plot()
    
    print("MSE = {}".format(mse(dt, res[:-1])))
    print("MAPE = {}".format(MAPE(dt, res[:-1])))


# ML modelling

In [ ]:
#
# main source: https://machinelearningmastery.com/xgboost-for-time-series-forecasting/
#
# doesn`t complete
# forecast monthly births with xgboost

# from numpy import asarray
# from pandas import read_csv
# from pandas import DataFrame
# from pandas import concat
# from sklearn.metrics import mean_absolute_error
# from xgboost import XGBRegressor
# from matplotlib import pyplot

# # transform a time series dataset into a supervised learning dataset
# def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
#     n_vars = 1
#     df = DataFrame(data)
#     cols = list()
#     # input sequence (t-n, ... t-1)
#     for i in range(n_in, 0, -1):
#         cols.append(df.shift(i))
#     # forecast sequence (t, t+1, ... t+n)
#     for i in range(0, n_out):
#         cols.append(df.shift(-i))
#     # put it all together
#     agg = concat(cols, axis=1)
#     # drop rows with NaN values
#     if dropnan:
#         agg.dropna(inplace=True)
#     return agg.values

# # fit an xgboost model and make a one step prediction
# def xgboost_forecast(train, testX):
#     # transform list into array
#     train = asarray(train)
#     # split into input and output columns
#     trainX, trainy = train[:, :-1], train[:, -1]
#     # fit model
#     model = XGBRegressor(max_depth=4, objective='reg:squarederror', n_estimators=1000)
#     model.fit(trainX, trainy)
#     # make a one-step prediction
#     yhat = model.predict(asarray([testX]), validate_features=True)
#     return yhat[0]

# # walk-forward validation for univariate data
# # def walk_forward_validation(data, n_test):
# #     predictions = list()
# #     # split dataset
# #     train, test = data[:-12], data[-12:]
# #     # seed history with training dataset
# #     history = [x for x in train]
# #     # step over each time-step in the test set
# #     for i in range(len(test)):
# #         # split test row into input and output columns
# #         testX, testy = test[i, :-1], test[i, -1]
# #         # fit model on history and make a prediction
# #         yhat = xgboost_forecast(history, testX)
# #         # store forecast in list of predictions
# #         predictions.append(yhat)
# #         # add actual observation to history for the next loop
# #         history.append(test[i])
# #         # summarize progress
# #         print('>expected=%.1f, predicted=%.1f' % (testy, yhat))
# #     # estimate prediction error
# #     error = mean_absolute_error(test[:, -1], predictions)
# #     return error, test[:, -1], predictions

# # load the dataset
# # series = DATA[1]
# # values = series.values
# # # transform the time series data into supervised learning
# # data = series_to_supervised(values, n_in=6)
# # # evaluate
# # mae, y, yhat = walk_forward_validation(data, 12)

# # print('MAE: %.3f' % mae)
# # # plot expected vs preducted
# # pyplot.plot(y, label='Expected')
# # pyplot.plot(yhat, label='Predicted')
# # pyplot.legend()
# # pyplot.show()
# # #-------------------------------
# predictions = list()
    
# history = [x for x in data]

# for i in range(12):
#     # split test row into input and output columns
#     testX, testy = test[i, :-1], test[i, -1]
#     # fit model on history and make a prediction
#     yhat = xgboost_forecast(history, testX)
#     # store forecast in list of predictions
#     predictions.append(yhat)
#     # add actual observation to history for the next loop
#     history.append(test[i])

# values = series.values
# # transform the time series data into supervised learning
# train = series_to_supervised(values, n_in=6)
# # split into input and output columns
# trainX, trainy = train[:, :-1], train[:, -1]
# # fit model
# model = XGBRegressor(objective='reg:squarederror', n_estimators=1000)
# model.fit(trainX, trainy)
# # construct an input for a new preduction
# row = values[-6:].flatten()
# # make a one-step prediction
# # yhat = model.predict(asarray([row]))
# # print('Input: %s, Predicted: %.3f' % (row, yhat[0]))
# plt.plot(predictions)


In [ ]:
# trainy.to_list()

In [ ]:
# series.keys()

In [ ]:
DATA[1].tail
# 

# FB Prophet

In [ ]:
from fbprophet.diagnostics import cross_validation

dt = (DATA[1].copy())

train_index = dt.index[:-12]
test_index = dt.index[-12:]
valid_index = dt.index[:-112]

train_val = dt.values[:-12]
test_val = dt.values[-12:]
valid_val = dt.values[:-112]

# office = office.rename(columns={'DatetimeIndex': 'ds', 'Sales': 'y'})

dt_model = Prophet(daily_seasonality=True,
                   seasonality_mode='additive')
dt_model.add_seasonality(name='monthly', period=4, fourier_order=5)

res_model = Prophet(daily_seasonality=True,
                   seasonality_mode='additive')
res_model.add_seasonality(name='monthly', period=4, fourier_order=5)

validate = pd.DataFrame()
validate["ds"] = valid_index
validate["y"] = valid_val
validate
validate["floor"] = validate['y'].min()
validate["cap"] = validate["y"].max() + (validate['y'].max() * .2)

dt_model.fit(validate)

In [ ]:
future = dt_model.make_future_dataframe(periods=12, include_history=True, freq="m")
future['floor'] = validate['y'].min()
future['cap'] = validate['y'].max() + (validate['y'].max() *0.2)

forecast = dt_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

fig1 = dt_model.plot(forecast)
DATA[1][:-100].plot(color='g')

#### On the validation data (first 11 years) the model shows very good results. what about a full dataset?

In [ ]:
train = pd.DataFrame()
train["ds"] = train_index
train["y"] = train_val
train
train["floor"] = train['y'].min()
train["cap"] = train["y"].max() + (train['y'].max() * .2)

res_model.fit(train)

future = res_model.make_future_dataframe(periods=12, include_history=True, freq="m")
future['floor'] = train['y'].min()
future['cap'] = train['y'].max() + (train['y'].max() *0.2)

forecast = dt_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

fig1 = res_model.plot(forecast)
DATA[1].plot(color='g')
plt.grid()
# plt.plot(time[:-12], train, 'b--', label="train")
plt.plot(test_index, test_val, color='orange', linestyle="--", label="test")

#### Not so good, but still beautiful. what about metrics?


In [ ]:
print(res_model.plot_components(forecast))

In [ ]:
forecast

In [ ]:
res = cross_validation(model=res_model, horizon="365 days")

In [ ]:
df_p = performance_metrics(res)
df_p.head()

# Deep learning approaches

nothing better proposed approach

In [ ]:
values = np.concatenate((DATA[0].values, DATA[1].values.reshape((-1, 1))), axis=1)
scaler = MinMaxScaler(feature_range=(0, 1))
values = scaler.fit_transform(values)

VALIDATION_SHIFT = 12
EPOCHS = 200

train = values[:-VALIDATION_SHIFT, :]
test = values[-VALIDATION_SHIFT:, :]

# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1]
test_X, test_y = test[:, :-1], test[:, -1]

optimizer = Adagrad(0.01)

# reshape input to be 3D [samples, features, timesteps]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))

model = Sequential()
model.add(LSTM(20, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dropout(0.2))
model.add(Dense(1, activation='relu'))
# model.compile(loss='mae', optimizer=optimizer)
model.compile(loss='mean_squared_error', optimizer='adam')

# fit network
history = model.fit(train_X, train_y, 
                    epochs=EPOCHS, batch_size=64, 
                    validation_data=(test_X, test_y), 
                    verbose=2, shuffle=False)

# plot history
plt.plot(history.history['loss'], label='train')
plt.plot(history.history['val_loss'], label='test')
plt.legend()
plt.show()

# make a prediction
yhat = model.predict(test_X)
yhat[yhat<0] = 0
yhat_train = model.predict(train_X)
yhat_train[yhat_train<0] = 0

test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
train_X = train_X.reshape((train_X.shape[0], train_X.shape[2]))



In [ ]:
# invert scaling for training
inv_yhat_train = np.concatenate((train_X, yhat_train), axis=1)
inv_yhat_train = scaler.inverse_transform(inv_yhat_train)
inv_yhat_train = inv_yhat_train[:,-1]


# invert scaling for forecast
inv_yhat = np.concatenate((test_X, yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,-1]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,-1]


# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
print("Actual data: ", inv_y)
print("Forecast: ",inv_yhat)

plt.plot(pd.Series(inv_yhat_train, index=DATA[1].index[:-VALIDATION_SHIFT]), label="Trained forecast")
plt.plot(DATA[1], label="Actual data")
plt.plot(pd.Series(inv_yhat, index=DATA[1].index[-VALIDATION_SHIFT:]), label="Forecast")
plt.legend()

In [ ]:
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1], 1))
test_X = test_X.reshape((test_X.shape[0], test_X.shape[1], 1))

model_cnn = Sequential()
model_cnn.add(Conv1D(filters=64, kernel_size=3, activation='relu', 
                     input_shape=(train_X.shape[1], train_X.shape[2])))
model_cnn.add(MaxPooling1D(pool_size=2))
model_cnn.add(Flatten())
model_cnn.add(Dense(50, activation='relu'))
model_cnn.add(Dense(1))
model_cnn.compile(loss='mse', optimizer=optimizer)

history = model_cnn.fit(train_X, train_y, 
                        epochs=EPOCHS, batch_size=64, 
                        validation_data=(test_X, test_y), 
                        verbose=2, shuffle=False)

# plot history
plt.plot(history.history['loss'][1:], label='train')
plt.plot(history.history['val_loss'][1:], label='test')
plt.legend()
plt.show()

# make a prediction
yhat = model_cnn.predict(test_X)
yhat[yhat<0] = 0
yhat_train = model_cnn.predict(train_X)
yhat_train[yhat_train<0] = 0

test_X = test_X.reshape((test_X.shape[0], test_X.shape[1]))
train_X = train_X.reshape((train_X.shape[0], train_X.shape[1]))

In [ ]:
# invert scaling for training
inv_yhat_train = np.concatenate((train_X, yhat_train), axis=1)
inv_yhat_train = scaler.inverse_transform(inv_yhat_train)
inv_yhat_train = inv_yhat_train[:,-1]


# invert scaling for forecast
inv_yhat = np.concatenate((test_X, yhat), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,-1]

# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = np.concatenate((test_X, test_y), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,-1]


# calculate RMSE
rmse = np.sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)
print("Actual data: ", inv_y)
print("Forecast: ",inv_yhat)

plt.plot(pd.Series(inv_yhat_train, index=DATA[1].index[:-VALIDATION_SHIFT]), label="Trained forecast")
plt.plot(DATA[1], label="Actual data")
plt.plot(pd.Series(inv_yhat, index=DATA[1].index[-VALIDATION_SHIFT:]), label="Forecast")
plt.legend()

# Conclusion & prediction for one year

I tested all models (except ML) and in my opinion, even ARIMA and fbprophet are very good approaches, and ARIMA fit's the model even better then fb, but for long term predictions it is better to use fb.

My series is additive. 

About stationary series - the best way is to use EMA and sub from the data the series. 

Differencing is not very good approach in this case.

In [ ]:
from fbprophet.diagnostics import cross_validation

dt = (DATA[1].copy())[150:]

res_model = Prophet(daily_seasonality=True,
                   seasonality_mode='additive')
res_model.add_seasonality(name='monthly', period=4, fourier_order=5)

train = pd.DataFrame()
train["ds"] = dt.index
train["y"] = dt.values

train["floor"] = train['y'].min()
train["cap"] = train["y"].max() + (train['y'].max() * .2)

res_model.fit(train)

future = res_model.make_future_dataframe(periods=12, include_history=True, freq="m")
future['floor'] = train['y'].min()
future['cap'] = train['y'].max() + (train['y'].max() *0.2)

forecast = dt_model.predict(future)
forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

fig1 = res_model.plot(forecast)
dt.plot(color='g')
plt.grid()